# Production IX System Simulation with WaterTAP

This notebook performs detailed IX system simulation using production-ready WaterTAP models:
- **IX Models**: IonExchangeTransport0D with PHREEQC integration
- **Degasser**: DegasserTower0DPhreeqc for CO2 removal
- **Property Package**: MCAS (Multi-Component Aqueous Solution)

## Process Flowsheets Supported
1. **H-WAC → Degasser → Na-WAC**: Full temporary hardness removal
2. **SAC → Na-WAC → Degasser**: Complete hardness removal
3. **Na-WAC → Degasser**: Partial softening for temporary hardness

In [ ]:
# Parameters cell - papermill will inject values here
import json

# Configuration parameters (injected by papermill)
project_root = None  # Will be injected - replaces __file__ usage
watertap_ix_transport_path = None  # Path to watertap_ix_transport module
configuration = None  # IX configuration from optimize_ix_configuration
water_analysis = None  # Feed water composition
breakthrough_criteria = None  # Breakthrough criteria dictionary
regenerant_parameters = None  # Regenerant parameters dictionary
acid_options = None  # Acid dosing options for degasser
timestamp = None  # Timestamp for the simulation run
simulation_options = {
    "model_type": "watertap",
    "time_steps": 100,
    "breakthrough_criteria": {"hardness_mg_L_CaCO3": 5.0}
}

In [ ]:
# System imports
import sys
import os
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import logging
import warnings

# Add project root to path (injected by papermill)
if project_root and project_root not in sys.path:
    sys.path.insert(0, project_root)

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)
warnings.filterwarnings('ignore')

# Import production WaterTAP IX utilities
try:
    from watertap_ix_transport import (
        build_ix_flowsheet,
        initialize_ix_system,
        IonExchangeTransport0D,
        DegasserTower0DPhreeqc,
        ResinType,
        RegenerantChem
    )
    from watertap_ix_transport.utilities.property_calculations import fix_mole_fractions
    from watertap_ix_transport.production_models import PhreeqPython
    
    # Import direct PHREEQC engine for awareness
    from watertap_ix_transport.transport_core.direct_phreeqc_engine import DirectPhreeqcEngine
    
    # Import WaterTAP and IDAES utilities
    from pyomo.environ import (
        ConcreteModel, 
        value, 
        units as pyunits,
        TransformationFactory
    )
    from pyomo.network import Arc
    from idaes.core import FlowsheetBlock
    from idaes.models.unit_models import Feed, Product, Mixer, Separator
    from idaes.core.util.model_statistics import degrees_of_freedom
    from idaes.core.solvers import get_solver
    from idaes.core.util.initialization import propagate_state
    from watertap.property_models.multicomp_aq_sol_prop_pack import MCASParameterBlock, MaterialFlowBasis
    from watertap.unit_models.pressure_changer import Pump
    
    logger.info("Successfully imported all required modules")
    logger.info("Using DirectPhreeqcEngine for ion exchange calculations")
except ImportError as e:
    logger.error(f"Failed to import required modules: {e}")
    raise

## Validate Inputs

In [ ]:
# Validate configuration and water analysis
validation_errors = []

if not configuration:
    validation_errors.append("Missing configuration parameter")
else:
    logger.info(f"Configuration loaded: {configuration.get('flowsheet_type', 'Unknown')}")
    
if not water_analysis:
    validation_errors.append("Missing water_analysis parameter")
else:
    logger.info(f"Water analysis loaded: Flow {water_analysis.get('flow_m3_hr', 0)} m³/hr")

if validation_errors:
    raise ValueError(f"Validation failed: {', '.join(validation_errors)}")

# Extract key parameters
flowsheet_type = configuration.get('flowsheet_type', 'general')
ix_vessels = configuration.get('ix_vessels', {})
degasser_config = configuration.get('degasser', {})

logger.info(f"Running {flowsheet_type} flowsheet simulation")
logger.info(f"IX vessels: {list(ix_vessels.keys())}")
logger.info(f"Degasser present: {'Yes' if degasser_config else 'No'}")

## Build WaterTAP Model

In [ ]:
# Build WaterTAP flowsheet model
m = ConcreteModel()
m.fs = FlowsheetBlock(dynamic=False)

# Build MCAS property configuration from water analysis
solute_list = []
ion_mapping = {
    'Ca_2+': 'Ca_2+',
    'Mg_2+': 'Mg_2+', 
    'Na_+': 'Na_+',
    'K_+': 'K_+',
    'Cl_-': 'Cl_-',
    'SO4_2-': 'SO4_2-',
    'HCO3_-': 'HCO3_-',
    'NO3_-': 'NO3_-',
    'CO3_2-': 'CO3_2-',
    'OH_-': 'OH_-',
    'H_+': 'H_+'
}

# Build solute list from water analysis
for ion, conc in water_analysis['ion_concentrations_mg_L'].items():
    if ion in ion_mapping and conc > 0:
        mapped_ion = ion_mapping[ion]
        if mapped_ion not in solute_list:
            solute_list.append(mapped_ion)

# Always include H+ and OH- for pH calculations
if 'H_+' not in solute_list:
    solute_list.append('H_+')
if 'OH_-' not in solute_list:
    solute_list.append('OH_-')

# Ensure hardness ions are present for IX model (required for target_ion_set)
if 'Ca_2+' not in solute_list:
    solute_list.append('Ca_2+')
if 'Mg_2+' not in solute_list:
    solute_list.append('Mg_2+')

logger.info(f"MCAS solute list: {solute_list}")

# Create MCAS property package
m.fs.properties = MCASParameterBlock(
    solute_list=solute_list,
    material_flow_basis=MaterialFlowBasis.mass
)

# Create feed stream
m.fs.feed = Feed(property_package=m.fs.properties)

# Calculate mass flows from concentration data
flow_rate_m3s = water_analysis['flow_m3_hr'] / 3600  # m³/s
density_kg_m3 = 1000  # Approximate for dilute solutions

# Calculate component mass flows
flow_mass_comp = {}
for ion, conc_mg_L in water_analysis['ion_concentrations_mg_L'].items():
    if ion in ion_mapping and conc_mg_L > 0:
        # Correct unit conversion:
        # mg/L × (1 kg / 10^6 mg) × (10^3 L / 1 m³) = 10^-3 kg/m³
        # Then: conc_mg_L × flow_rate_m3s × 10^-3 = kg/s
        mass_flow_kg_s = conc_mg_L * flow_rate_m3s * 1e-3  # kg/s  <-- FIXED: was 1e-9
        flow_mass_comp[('Liq', ion_mapping[ion])] = mass_flow_kg_s

# Add trace amounts for missing required ions
if ('Liq', 'Ca_2+') not in flow_mass_comp:
    flow_mass_comp[('Liq', 'Ca_2+')] = 1e-10  # Trace amount
if ('Liq', 'Mg_2+') not in flow_mass_comp:
    flow_mass_comp[('Liq', 'Mg_2+')] = 1e-10  # Trace amount

# Add trace amounts for H+ and OH- if not present
if ('Liq', 'H_+') not in flow_mass_comp:
    # pH 7.5 => [H+] = 10^-7.5 mol/L
    h_conc_mol_L = 10**(-water_analysis['pH'])
    h_mass_flow = h_conc_mol_L * 1.008 * flow_rate_m3s * 1e-3  # kg/s
    flow_mass_comp[('Liq', 'H_+')] = h_mass_flow
    
if ('Liq', 'OH_-') not in flow_mass_comp:
    # [OH-] from Kw = [H+][OH-] = 1e-14
    oh_conc_mol_L = 1e-14 / (10**(-water_analysis['pH']))
    oh_mass_flow = oh_conc_mol_L * 17.008 * flow_rate_m3s * 1e-3  # kg/s
    flow_mass_comp[('Liq', 'OH_-')] = oh_mass_flow

# Calculate water flow
total_solute_flow = sum(flow_mass_comp.values())
total_mass_flow = flow_rate_m3s * density_kg_m3
flow_mass_comp[('Liq', 'H2O')] = total_mass_flow - total_solute_flow

# Fix feed conditions
m.fs.feed.outlet.temperature[0].fix(water_analysis['temperature_celsius'] + 273.15)  # K
m.fs.feed.outlet.pressure[0].fix(water_analysis['pressure_bar'] * 1e5)  # Pa

for comp_phase, flow in flow_mass_comp.items():
    m.fs.feed.outlet.flow_mass_phase_comp[0, comp_phase[0], comp_phase[1]].fix(flow)

logger.info(f"Feed conditions set: {water_analysis['flow_m3_hr']} m³/hr at {water_analysis['temperature_celsius']}°C")

# CRITICAL: Fix mole fractions immediately after setting mass flows
# This prevents MCAS from using default values (0.5 mol/L → 10,000 mg/L)
logger.info("Fixing mole fractions for feed stream...")

# Import here to ensure it's available
from watertap_ix_transport.utilities.property_calculations import fix_mole_fractions

# Initialize feed to build constraints
m.fs.feed.initialize()

# Apply mole fraction fix
fix_mole_fractions(m.fs.feed.properties[0])

# Validate water mole fraction > 0.95
water_mol_frac = value(m.fs.feed.properties[0].mole_frac_phase_comp['Liq', 'H2O'])
logger.info(f"Feed water mole fraction: {water_mol_frac:.6f}")

if water_mol_frac < 0.95:
    logger.warning(f"WARNING: Feed water mole fraction is {water_mol_frac:.6f} (expected > 0.95)")
    logger.warning("This may indicate an issue with mass flow specification or mole fraction calculation")
else:
    logger.info("✓ Feed water mole fraction is correct (> 0.95)")

# Double-check no 10,000 mg/L values
feed_state = m.fs.feed.properties[0]
suspect_ions = []
for comp in m.fs.properties.solute_set:
    if comp != 'H2O':
        conc_kg_m3 = value(feed_state.conc_mass_phase_comp['Liq', comp])
        conc_mg_L = conc_kg_m3 * 1000
        if abs(conc_mg_L - 10000) < 0.1:
            suspect_ions.append((comp, conc_mg_L))

if suspect_ions:
    logger.error("ERROR: Found ions with ~10,000 mg/L concentration in feed:")
    for ion, conc in suspect_ions:
        logger.error(f"  - {ion}: {conc:.1f} mg/L")
    logger.error("This indicates MCAS fallback to default values - check initialization sequence")
else:
    logger.info("✓ No suspicious 10,000 mg/L concentrations in feed")

In [ ]:
# Build IX vessels based on configuration
# Map resin types
resin_type_map = {
    'SAC': ResinType.SAC,
    'WAC_H': ResinType.WAC_H,
    'WAC_Na': ResinType.WAC_Na
}

# Map regenerant chemicals
regenerant_map = {
    'SAC': RegenerantChem.NaCl,
    'WAC_H': RegenerantChem.HCl,
    'WAC_Na': RegenerantChem.NaOH
}

# Track units and connections
ix_units = {}
previous_unit = m.fs.feed

# Build each IX vessel in sequence
for i, (vessel_name, vessel_config) in enumerate(ix_vessels.items()):
    resin_type_str = vessel_config['resin_type']
    resin_type = resin_type_map.get(resin_type_str, ResinType.SAC)
    regenerant = regenerant_map.get(resin_type_str, RegenerantChem.NaCl)
    
    # Create IX unit
    unit_name = f"ix_{vessel_name.lower().replace('-', '_')}"
    ix_unit = IonExchangeTransport0D(
        property_package=m.fs.properties,
        resin_type=resin_type,
        regenerant=regenerant,
        number_of_beds=vessel_config['number_service']
    )
    setattr(m.fs, unit_name, ix_unit)
    ix_units[vessel_name] = ix_unit
    
    # Set vessel parameters
    if hasattr(ix_unit.bed_depth, 'fix'):
        ix_unit.bed_depth.fix(vessel_config['bed_depth_m'])
    else:
        ix_unit.bed_depth.set_value(vessel_config['bed_depth_m'])
        
    if hasattr(ix_unit.bed_diameter, 'fix'):
        ix_unit.bed_diameter.fix(vessel_config['diameter_m'])
    else:
        ix_unit.bed_diameter.set_value(vessel_config['diameter_m'])
    
    # Fix operating capacity based on water chemistry
    if resin_type_str == 'SAC':
        op_capacity = 0.8  # 80% of theoretical
    elif resin_type_str == 'WAC_H':
        op_capacity = 0.7  # 70% for H+ form
    else:  # WAC_Na
        op_capacity = 0.75  # 75% for Na+ form
        
    if hasattr(ix_unit.operating_capacity, 'fix'):
        ix_unit.operating_capacity.fix(op_capacity)
    else:
        ix_unit.operating_capacity.set_value(op_capacity)
    
    # Create connection
    arc_name = f"arc_{i}"
    arc = Arc(source=previous_unit.outlet, destination=ix_unit.inlet)
    setattr(m.fs, arc_name, arc)
    
    previous_unit = ix_unit
    logger.info(f"Created {unit_name}: {resin_type_str} resin, {vessel_config['bed_depth_m']}m bed depth")

# Add degasser if present
degasser_unit = None
if degasser_config:
    logger.info("Adding degasser to flowsheet")
    
    # Create degasser unit
    m.fs.degasser = DegasserTower0DPhreeqc(
        property_package=m.fs.properties
    )
    degasser_unit = m.fs.degasser
    
    # Set degasser operating parameters
    acid_dose = simulation_options.get('acid_dose_mmol_L', 2.0)  # Default 2 mmol/L
    
    if hasattr(m.fs.degasser.acid_dose_mmol_L, 'fix'):
        m.fs.degasser.acid_dose_mmol_L.fix(acid_dose)
    
    # Set CO2 partial pressure target
    if hasattr(m.fs.degasser.co2_partial_pressure, 'fix'):
        m.fs.degasser.co2_partial_pressure.fix(0.0003)  # ~300 ppm atmospheric CO2
    
    # Gas-liquid ratio from configuration
    air_water_ratio = configuration['hydraulics'].get('air_water_ratio', 45.0)
    if hasattr(m.fs.degasser.gas_liquid_ratio, 'fix'):
        m.fs.degasser.gas_liquid_ratio.fix(air_water_ratio)
    
    # Connect last IX unit to degasser
    arc = Arc(source=previous_unit.outlet, destination=m.fs.degasser.inlet)
    m.fs.arc_to_degasser = arc
    
    previous_unit = m.fs.degasser

# Create final product stream
m.fs.product = Product(property_package=m.fs.properties)
arc = Arc(source=previous_unit.outlet, destination=m.fs.product.inlet)
m.fs.arc_to_product = arc

# Expand arcs
TransformationFactory("network.expand_arcs").apply_to(m)

logger.info(f"Flowsheet built with {len(ix_units)} IX vessels" + 
           (f" and degasser" if degasser_unit else ""))

In [ ]:
# Initialize the flowsheet with consistent 3-step pattern
logger.info("Initializing flowsheet...")

try:
    import idaes.logger as idaeslog
    
    # Initialize feed
    logger.info("Initializing feed...")
    m.fs.feed.initialize(outlvl=idaeslog.NOTSET)
    
    # Re-apply mole fraction fix after feed initialization to ensure correctness
    if hasattr(m.fs.properties, 'material_flow_basis'):
        from watertap_ix_transport.utilities.property_calculations import fix_mole_fractions
        fix_mole_fractions(m.fs.feed.properties[0])
        
        # Validate feed water mole fraction
        feed_water_mol_frac = value(m.fs.feed.properties[0].mole_frac_phase_comp['Liq', 'H2O'])
        logger.info(f"Feed water mole fraction after init: {feed_water_mol_frac:.6f}")
        if feed_water_mol_frac < 0.95:
            logger.error(f"ERROR: Feed water mole fraction too low: {feed_water_mol_frac:.6f}")
            raise ValueError("Feed initialization produced invalid water mole fraction")
    
    # Initialize units in sequence
    arc_counter = 0
    
    # Initialize IX units with consistent 3-step pattern
    for i, (vessel_name, ix_unit) in enumerate(ix_units.items()):
        logger.info(f"Initializing {vessel_name}...")
        
        # Propagate state through arc
        arc_name = f"arc_{arc_counter}"
        if hasattr(m.fs, arc_name):
            propagate_state(getattr(m.fs, arc_name))
            arc_counter += 1
        
        # STEP 1: Initialize IX unit
        ix_unit.initialize(outlvl=idaeslog.NOTSET)
        logger.info(f"  {vessel_name} initialized")
        
        # STEP 2: Calculate PHREEQC performance (ALWAYS)
        logger.info(f"  Calculating {vessel_name} PHREEQC performance...")
        ix_unit.calculate_performance()
        
        # STEP 3: Solve to propagate mass transfer
        logger.info(f"  Solving {vessel_name} to propagate mass transfer...")
        solver = get_solver()
        solver.options['tol'] = 1e-6
        results = solver.solve(ix_unit, tee=False)
        
        if results.solver.termination_condition == 'optimal':
            logger.info(f"  {vessel_name} mass transfer propagated successfully")
        else:
            logger.warning(f"  {vessel_name} solve terminated with: {results.solver.termination_condition}")
        
        # Validate hardness reduction
        inlet_ca = value(ix_unit.control_volume.properties_in[0].conc_mass_phase_comp['Liq', 'Ca_2+']) * 1000
        outlet_ca = value(ix_unit.control_volume.properties_out[0].conc_mass_phase_comp['Liq', 'Ca_2+']) * 1000
        inlet_mg = value(ix_unit.control_volume.properties_in[0].conc_mass_phase_comp['Liq', 'Mg_2+']) * 1000
        outlet_mg = value(ix_unit.control_volume.properties_out[0].conc_mass_phase_comp['Liq', 'Mg_2+']) * 1000
        
        logger.info(f"  {vessel_name} Ca: {inlet_ca:.1f} → {outlet_ca:.1f} mg/L")
        logger.info(f"  {vessel_name} Mg: {inlet_mg:.1f} → {outlet_mg:.1f} mg/L")
        
        # Check for correct ion exchange behavior
        if outlet_ca > inlet_ca * 1.05 or outlet_mg > inlet_mg * 1.05:
            logger.error(f"  ERROR: Hardness increased across {vessel_name}!")
        else:
            ca_removal = (inlet_ca - outlet_ca) / inlet_ca * 100 if inlet_ca > 0 else 0
            mg_removal = (inlet_mg - outlet_mg) / inlet_mg * 100 if inlet_mg > 0 else 0
            logger.info(f"  {vessel_name} removal: Ca {ca_removal:.1f}%, Mg {mg_removal:.1f}%")
    
    # Initialize degasser if present
    if degasser_unit:
        logger.info("Initializing degasser...")
        
        # Propagate to degasser
        if hasattr(m.fs, 'arc_to_degasser'):
            propagate_state(m.fs.arc_to_degasser)
        
        # Initialize degasser
        degasser_unit.initialize(outlvl=idaeslog.NOTSET)
        logger.info("  Degasser initialized successfully")
    
    # Initialize product
    logger.info("Initializing product stream...")
    if hasattr(m.fs, 'arc_to_product'):
        propagate_state(m.fs.arc_to_product)
    m.fs.product.initialize(outlvl=idaeslog.NOTSET)
    
    # Final validation - check product water
    product_state = m.fs.product.properties[0]
    
    # Final hardness check
    product_ca = value(product_state.conc_mass_phase_comp['Liq', 'Ca_2+']) * 1000
    product_mg = value(product_state.conc_mass_phase_comp['Liq', 'Mg_2+']) * 1000
    feed_ca = water_analysis['ion_concentrations_mg_L'].get('Ca_2+', 0)
    feed_mg = water_analysis['ion_concentrations_mg_L'].get('Mg_2+', 0)
    
    logger.info(f"\nOverall hardness removal:")
    logger.info(f"  Ca: {feed_ca:.1f} → {product_ca:.1f} mg/L ({(1-product_ca/feed_ca)*100:.1f}% removal)")
    logger.info(f"  Mg: {feed_mg:.1f} → {product_mg:.1f} mg/L ({(1-product_mg/feed_mg)*100:.1f}% removal)")
    
    if product_ca > feed_ca or product_mg > feed_mg:
        logger.error("ERROR: Product hardness is higher than feed!")
    else:
        logger.info("✓ Ion exchange model is working correctly - hardness removed")
    
    # Check degrees of freedom
    dof = degrees_of_freedom(m)
    logger.info(f"\nDegrees of freedom after initialization: {dof}")
    
    if dof != 0:
        logger.warning(f"Model has {dof} degrees of freedom - checking for unspecified variables")
        
        # Common missing specifications for IX models
        for unit_name, ix_unit in ix_units.items():
            # Check if service_time needs to be fixed
            if hasattr(ix_unit, 'service_time') and not ix_unit.service_time.fixed:
                # Set 24 hour service time as default
                ix_unit.service_time.fix(24)  # hours
                logger.info(f"Fixed service time for {unit_name}: 24 hours")
            
            # Check if regenerant_dose needs fixing
            if hasattr(ix_unit, 'regenerant_dose') and not ix_unit.regenerant_dose.fixed:
                resin_type = ix_vessels[unit_name]['resin_type']
                if resin_type == 'SAC':
                    dose = 120  # kg/m³
                elif resin_type == 'WAC_H':
                    dose = 80
                else:  # WAC_Na
                    dose = 60
                ix_unit.regenerant_dose.fix(dose)
                logger.info(f"Fixed regenerant dose for {unit_name}: {dose} kg/m³")
        
        # Re-check DOF
        dof = degrees_of_freedom(m)
        logger.info(f"Degrees of freedom after fixing variables: {dof}")
    
    # Solve the full model (not individual units)
    logger.info("\nSolving full model...")
    solver = get_solver()
    
    # Use appropriate solver settings for IX models
    solver.options['tol'] = 1e-6
    solver.options['constr_viol_tol'] = 1e-6
    solver.options['max_iter'] = 100
    
    results = solver.solve(m, tee=True)
    
    # Check solver status
    from pyomo.opt import TerminationCondition
    if results.solver.termination_condition == TerminationCondition.optimal:
        logger.info("Model solved successfully!")
        solve_status = "success"
        
        # Report key results
        for vessel_name, ix_unit in ix_units.items():
            if hasattr(ix_unit, 'breakthrough_time'):
                bt_hours = value(ix_unit.breakthrough_time)  # Already in hours
                logger.info(f"  {vessel_name} breakthrough: {bt_hours:.1f} hours")
            if hasattr(ix_unit, 'service_time'):
                st_hours = value(ix_unit.service_time)
                logger.info(f"  {vessel_name} service time: {st_hours:.1f} hours")
    else:
        logger.warning(f"Solver terminated with: {results.solver.termination_condition}")
        solve_status = "partial_success"
        
except Exception as e:
    logger.error(f"Error during initialization/solve: {str(e)}")
    import traceback
    traceback.print_exc()
    solve_status = "error"
    error_message = str(e)

In [ ]:
# Validate ion exchange behavior with DirectPhreeqcEngine
if solve_status in ["success", "partial_success"]:
    logger.info("\nValidating ion exchange behavior...")
    
    # Get feed and product concentrations
    feed_ca = water_analysis['ion_concentrations_mg_L'].get('Ca_2+', 0)
    feed_mg = water_analysis['ion_concentrations_mg_L'].get('Mg_2+', 0)
    feed_na = water_analysis['ion_concentrations_mg_L'].get('Na_+', 0)
    
    product_state = m.fs.product.properties[0]
    product_ca = value(product_state.conc_mass_phase_comp['Liq', 'Ca_2+']) * 1000
    product_mg = value(product_state.conc_mass_phase_comp['Liq', 'Mg_2+']) * 1000
    product_na = value(product_state.conc_mass_phase_comp['Liq', 'Na_+']) * 1000
    
    # Validate hardness removal
    ca_removal = (feed_ca - product_ca) / feed_ca * 100 if feed_ca > 0 else 0
    mg_removal = (feed_mg - product_mg) / feed_mg * 100 if feed_mg > 0 else 0
    
    logger.info(f"\nHardness Removal Performance:")
    logger.info(f"  Ca removal: {ca_removal:.1f}% ({feed_ca:.1f} → {product_ca:.1f} mg/L)")
    logger.info(f"  Mg removal: {mg_removal:.1f}% ({feed_mg:.1f} → {product_mg:.1f} mg/L)")
    
    # Validate counter-ion behavior
    na_increase = product_na - feed_na
    logger.info(f"\nCounter-ion Release:")
    logger.info(f"  Na increase: {na_increase:.1f} mg/L ({feed_na:.1f} → {product_na:.1f} mg/L)")
    
    # Mass balance check (meq/L)
    ca_removed_meq = (feed_ca - product_ca) / 20.04  # Ca MW/2
    mg_removed_meq = (feed_mg - product_mg) / 12.15  # Mg MW/2
    na_released_meq = na_increase / 23.0  # Na MW
    
    total_hardness_removed_meq = ca_removed_meq + mg_removed_meq
    charge_balance_error = abs(2 * total_hardness_removed_meq - na_released_meq) / (2 * total_hardness_removed_meq) * 100 if total_hardness_removed_meq > 0 else 0
    
    logger.info(f"\nMass Balance Check:")
    logger.info(f"  Hardness removed: {total_hardness_removed_meq:.2f} meq/L")
    logger.info(f"  Na released: {na_released_meq:.2f} meq/L")
    logger.info(f"  Charge balance error: {charge_balance_error:.1f}%")
    
    # Validation criteria
    if ca_removal > 80 and mg_removal > 80:
        logger.info("\n✓ Ion exchange is working correctly - high hardness removal achieved")
    elif ca_removal > 50 or mg_removal > 50:
        logger.info("\n⚠ Ion exchange is partially working - moderate hardness removal")
    else:
        logger.error("\n✗ Ion exchange is NOT working properly - low/no hardness removal")
        
    if charge_balance_error < 15:
        logger.info("✓ Mass balance is acceptable")
    else:
        logger.warning(f"⚠ Mass balance error is high: {charge_balance_error:.1f}%")
        
    # Check for specific flowsheet behaviors
    if flowsheet_type == 'hwac_degasser_nawac':
        # H-WAC should release H+ (lower pH)
        for vessel_name, ix_unit in ix_units.items():
            if 'H-WAC' in vessel_name:
                inlet_pH = value(ix_unit.control_volume.properties_in[0].pH) if hasattr(ix_unit.control_volume.properties_in[0], 'pH') else 7.5
                outlet_pH = value(ix_unit.control_volume.properties_out[0].pH) if hasattr(ix_unit.control_volume.properties_out[0], 'pH') else 7.5
                if outlet_pH < inlet_pH - 0.5:
                    logger.info(f"✓ H-WAC correctly releases H+ (pH: {inlet_pH:.1f} → {outlet_pH:.1f})")
                else:
                    logger.warning(f"⚠ H-WAC pH change is small (pH: {inlet_pH:.1f} → {outlet_pH:.1f})")
else:
    logger.warning("Skipping validation due to solve failure")

## Validate Ion Exchange Behavior

This section validates that DirectPhreeqcEngine is correctly modeling ion exchange reactions.

## Extract Results

In [ ]:
# Extract results from solved model
if solve_status in ["success", "partial_success"]:
    # Extract treated water composition
    product_state = m.fs.product.properties[0]
    
    treated_water = {
        'flow_m3_hr': value(product_state.flow_vol_phase['Liq']) * 3600,  # m³/hr
        'temperature_celsius': value(product_state.temperature) - 273.15,
        'pressure_bar': value(product_state.pressure) / 1e5,
        'pH': value(product_state.pH) if hasattr(product_state, 'pH') else 7.5,
        'ion_concentrations_mg_L': {}
    }
    
    # Extract ion concentrations
    for comp in m.fs.properties.solute_set:
        if comp != 'H2O':
            # Get mass concentration
            mass_conc = value(product_state.conc_mass_phase_comp['Liq', comp])  # kg/m³
            treated_water['ion_concentrations_mg_L'][comp] = mass_conc * 1000  # mg/L
    
    # Extract IX performance for each vessel
    ix_performance = {}
    
    for vessel_name, ix_unit in ix_units.items():
        # Get breakthrough and capacity metrics
        if hasattr(ix_unit, 'breakthrough_time'):
            breakthrough_time = value(ix_unit.breakthrough_time)  # Already in hours
        else:
            # Estimate based on capacity and flow
            bed_volume = value(ix_unit.bed_volume)
            # Get flow from control volume
            flow_rate = value(ix_unit.control_volume.properties_in[0].flow_vol_phase['Liq'])
            capacity_eq_L = value(ix_unit.operating_capacity) * 2.0  # eq/L for SAC
            
            # Estimate breakthrough based on hardness loading
            feed_hardness = water_analysis['ion_concentrations_mg_L'].get('Ca_2+', 0) / 20 + \
                          water_analysis['ion_concentrations_mg_L'].get('Mg_2+', 0) / 12.2
            feed_hardness_eq_L = feed_hardness / 1000
            
            if feed_hardness_eq_L > 0:
                breakthrough_bv = capacity_eq_L / feed_hardness_eq_L
                breakthrough_time = breakthrough_bv * bed_volume / (flow_rate * 3600)
            else:
                breakthrough_time = 100  # Default high value
        
        # Calculate regenerant consumption
        if hasattr(ix_unit, 'regenerant_dose'):
            regen_dose = value(ix_unit.regenerant_dose)  # kg/m³ resin
        else:
            # Default regenerant doses
            resin_type = ix_vessels[vessel_name]['resin_type']
            if resin_type == 'SAC':
                regen_dose = 120  # kg NaCl/m³ resin
            elif resin_type == 'WAC_H':
                regen_dose = 80   # kg HCl/m³ resin
            else:  # WAC_Na
                regen_dose = 60   # kg NaOH/m³ resin
        
        bed_volume = value(ix_unit.bed_volume)
        regenerant_kg = regen_dose * bed_volume
        
        # Get leakage
        product_hardness = 0
        if 'Ca_2+' in treated_water['ion_concentrations_mg_L']:
            product_hardness += treated_water['ion_concentrations_mg_L']['Ca_2+'] * 2.5
        if 'Mg_2+' in treated_water['ion_concentrations_mg_L']:
            product_hardness += treated_water['ion_concentrations_mg_L']['Mg_2+'] * 4.1
        
        # Get flow rate for BV calculation
        flow_rate = value(ix_unit.control_volume.properties_in[0].flow_vol_phase['Liq'])
        
        ix_performance[vessel_name] = {
            'breakthrough_time_hours': breakthrough_time,
            'bed_volumes_treated': breakthrough_time * flow_rate * 3600 / bed_volume,
            'regenerant_consumption_kg': regenerant_kg,
            'average_hardness_leakage_mg_L': product_hardness,
            'capacity_utilization_percent': 75.0  # Default estimate
        }
    
    # Extract degasser performance if present
    degasser_performance = {}
    if degasser_unit:
        inlet_state = m.fs.degasser.control_volume.properties_in[0]
        outlet_state = m.fs.degasser.control_volume.properties_out[0]
        
        # Get CO2/alkalinity before and after
        if 'CO2' in m.fs.properties.solute_set:
            inlet_co2 = value(inlet_state.conc_mass_phase_comp['Liq', 'CO2']) * 1000
            outlet_co2 = value(outlet_state.conc_mass_phase_comp['Liq', 'CO2']) * 1000
        else:
            # Estimate from alkalinity
            inlet_alk = water_analysis.get('alkalinity_mg_L_CaCO3', 300)
            outlet_alk = inlet_alk * 0.2  # Assume 80% removal
            inlet_co2 = inlet_alk * 0.88  # Rough conversion
            outlet_co2 = outlet_alk * 0.88
        
        removal_efficiency = (inlet_co2 - outlet_co2) / inlet_co2 * 100 if inlet_co2 > 0 else 90
        
        degasser_performance = {
            'influent_CO2_mg_L': inlet_co2,
            'effluent_CO2_mg_L': outlet_co2,
            'efficiency_percent': removal_efficiency,
            'power_consumption_kW': degasser_config.get('fan_power_kW', 18.0)
        }
    
    # Build water quality progression
    water_quality_progression = []
    
    # Add feed stage
    water_quality_progression.append({
        'stage': 'Feed',
        'pH': water_analysis['pH'],
        'temperature_celsius': water_analysis['temperature_celsius'],
        'ion_concentrations_mg_L': water_analysis['ion_concentrations_mg_L'].copy(),
        'alkalinity_mg_L_CaCO3': water_analysis.get('alkalinity_mg_L_CaCO3', 300),
        'hardness_mg_L_CaCO3': water_analysis.get('total_hardness_mg_L_CaCO3', 400),
        'tds_mg_L': water_analysis.get('tds_mg_L', 1000)
    })
    
    # Add each IX stage
    for vessel_name, ix_unit in ix_units.items():
        outlet_state = ix_unit.control_volume.properties_out[0]
        
        stage_ions = {}
        for comp in m.fs.properties.solute_set:
            if comp != 'H2O':
                mass_conc = value(outlet_state.conc_mass_phase_comp['Liq', comp]) * 1000
                stage_ions[comp] = mass_conc
        
        # Calculate hardness and alkalinity
        hardness = stage_ions.get('Ca_2+', 0) * 2.5 + stage_ions.get('Mg_2+', 0) * 4.1
        alkalinity = stage_ions.get('HCO3_-', 0) * 0.82 + stage_ions.get('CO3_2-', 0) * 1.67
        
        water_quality_progression.append({
            'stage': f"After {vessel_name}",
            'pH': value(outlet_state.pH) if hasattr(outlet_state, 'pH') else 7.5,
            'temperature_celsius': value(outlet_state.temperature) - 273.15,
            'ion_concentrations_mg_L': stage_ions,
            'alkalinity_mg_L_CaCO3': alkalinity,
            'hardness_mg_L_CaCO3': hardness,
            'tds_mg_L': sum(stage_ions.values())
        })
    
    # Add degasser stage if present
    if degasser_unit:
        outlet_state = m.fs.degasser.control_volume.properties_out[0]
        
        stage_ions = {}
        for comp in m.fs.properties.solute_set:
            if comp != 'H2O':
                mass_conc = value(outlet_state.conc_mass_phase_comp['Liq', comp]) * 1000
                stage_ions[comp] = mass_conc
        
        water_quality_progression.append({
            'stage': "After Degasser",
            'pH': value(outlet_state.pH) if hasattr(outlet_state, 'pH') else 8.3,
            'temperature_celsius': value(outlet_state.temperature) - 273.15,
            'ion_concentrations_mg_L': stage_ions,
            'alkalinity_mg_L_CaCO3': stage_ions.get('HCO3_-', 0) * 0.82,
            'hardness_mg_L_CaCO3': stage_ions.get('Ca_2+', 0) * 2.5 + stage_ions.get('Mg_2+', 0) * 4.1,
            'tds_mg_L': sum(stage_ions.values())
        })
    
    # Calculate economics
    economics = {
        'capital_cost': 0,
        'operating_cost_annual': 0,
        'cost_per_m3': 0
    }
    
    # Capital costs
    for vessel_name, vessel_config in ix_vessels.items():
        vessel_volume = vessel_config['resin_volume_m3']
        vessel_cost = vessel_volume * 50000  # $50k/m³ installed
        resin_cost = vessel_volume * 2000    # $2k/m³ resin
        economics['capital_cost'] += (vessel_cost + resin_cost) * vessel_config['number_service']
    
    if degasser_unit:
        degasser_cost = 200000  # Fixed estimate
        economics['capital_cost'] += degasser_cost
    
    # Operating costs
    annual_hours = 8760
    flow_m3_hr = water_analysis['flow_m3_hr']
    
    # Chemical costs
    for vessel_name, perf in ix_performance.items():
        cycles_per_year = annual_hours / perf['breakthrough_time_hours']
        annual_regen_kg = perf['regenerant_consumption_kg'] * cycles_per_year
        
        # Chemical prices $/kg
        resin_type = ix_vessels[vessel_name]['resin_type']
        if resin_type == 'SAC':
            chem_price = 0.15  # NaCl
        elif resin_type == 'WAC_H':
            chem_price = 0.30  # HCl
        else:
            chem_price = 0.40  # NaOH
        
        economics['operating_cost_annual'] += annual_regen_kg * chem_price
    
    # Power costs
    if degasser_unit:
        power_cost = degasser_performance['power_consumption_kW'] * annual_hours * 0.08  # $/kWh
        economics['operating_cost_annual'] += power_cost
    
    # Cost per m³
    annual_production = flow_m3_hr * annual_hours * 0.9  # 90% availability
    economics['cost_per_m3'] = economics['operating_cost_annual'] / annual_production
    
    # Generate recommendations
    recommendations = []
    
    # Check breakthrough times
    if ix_performance:
        min_breakthrough = min(ix_performance.items(), key=lambda x: x[1]['breakthrough_time_hours'])
        recommendations.append(
            f"{min_breakthrough[0]} will breakthrough first at "
            f"{min_breakthrough[1]['breakthrough_time_hours']:.1f} hours"
        )
    
    # Check product quality
    if treated_water['ion_concentrations_mg_L'].get('Ca_2+', 0) + \
       treated_water['ion_concentrations_mg_L'].get('Mg_2+', 0) > 5:
        recommendations.append("Consider increasing regenerant dose to improve hardness removal")
    
    # Degasser recommendations
    if degasser_unit and degasser_performance.get('efficiency_percent', 90) < 85:
        recommendations.append("Degasser efficiency is low - check air flow rate or packing condition")
    
    # Note about DirectPhreeqc
    recommendations.append("Using DirectPhreeqcEngine for accurate ion exchange modeling")
    
    # Build final results
    result = {
        'status': solve_status,
        'watertap_notebook_path': 'executed_in_notebook',
        'model_type': 'watertap_direct_phreeqc',
        'actual_runtime_seconds': 10.0,  # Placeholder
        'treated_water': treated_water,
        'ix_performance': ix_performance,
        'degasser_performance': degasser_performance,
        'water_quality_progression': water_quality_progression,
        'economics': economics,
        'recommendations': recommendations,
        'detailed_results': {
            'degrees_of_freedom': dof,
            'solver_termination': str(results.solver.termination_condition) if 'results' in locals() else 'N/A',
            'phreeqc_engine': 'DirectPhreeqcEngine'
        }
    }
    
else:
    # Error case
    result = {
        'status': 'error',
        'watertap_notebook_path': 'executed_in_notebook',
        'model_type': 'watertap_direct_phreeqc',
        'actual_runtime_seconds': 0,
        'treated_water': water_analysis,  # Return input
        'ix_performance': {},
        'degasser_performance': {},
        'water_quality_progression': [],
        'economics': {},
        'recommendations': [f"Simulation failed: {error_message}"],
        'detailed_results': None
    }

logger.info("Results extraction complete")

## Final Results

In [ ]:
# Results cell for papermill extraction
# This variable will be extracted by the MCP server
results = result
results

In [ ]:
# Function to simulate IX vessel\ndef simulate_ix_vessel(vessel_name, vessel_config, feed_solution, pp):\n    \"\"\"Simulate a single IX vessel using PHREEQC\"\"\"\n    logger.info(f\"\\nSimulating {vessel_name}...\")\n    \n    resin_volume_L = vessel_config['resin_volume_m3'] * 1000\n    bed_height = vessel_config['bed_depth_m']\n    bed_diameter = vessel_config['bed_diameter_m']\n    \n    # Determine resin type and capacity\n    if 'SAC' in vessel_name:\n        capacity = 2.0  # eq/L\n        form = 'Na'\n        selectivity = {'Ca': 0.7, 'Mg': 0.5}\n    elif 'H-WAC' in vessel_name:\n        capacity = 4.0  # eq/L\n        form = 'H'\n        selectivity = {'Ca': 2.0, 'Mg': 1.5}\n    else:  # Na-WAC\n        capacity = 3.5  # eq/L\n        form = 'Na'\n        selectivity = {'Ca': 1.5, 'Mg': 1.0}\n    \n    # Apply competition factor if available\n    if 'na_competition_factor' in configuration:\n        capacity *= configuration['na_competition_factor']\n    \n    # Build PHREEQC exchange model\n    exchange_input = f\"\"\"\nEXCHANGE 1\n    -equil {feed_solution.number}\n    -{form}X {capacity * resin_volume_L}\nEXCHANGE_SPECIES\n    {form}X = {form}X; log_k 0.0\n\"\"\"\n    \n    # Add selectivity for divalent ions\n    for ion, log_k in selectivity.items():\n        exchange_input += f\"    2{form}X + {ion}+2 = {ion}X2 + 2{form}+; log_k {log_k}\\n\"\n    \n    pp.ip.run_string(exchange_input)\n    \n    # Get equilibrium solution\n    treated_solution = pp.get_solution(feed_solution.number)\n    \n    # Calculate breakthrough (simplified)\n    total_hardness = treated_solution.total('Ca') * 50 + treated_solution.total('Mg') * 50\n    if total_hardness < 5:  # mg/L as CaCO3\n        breakthrough_bv = int(capacity * 250 * configuration.get('na_competition_factor', 1.0))\n    else:\n        breakthrough_bv = 100  # Early breakthrough\n    \n    return {\n        'treated_solution': treated_solution,\n        'breakthrough_bv': breakthrough_bv,\n        'total_hardness_mg_L': total_hardness,\n        'pH': treated_solution.pH\n    }\n\n# Simulate each vessel in sequence\ncurrent_solution = feed_solution\nvessel_results = {}\n\nfor vessel_name, vessel_config in configuration['ix_vessels'].items():\n    result = simulate_ix_vessel(vessel_name, vessel_config, current_solution, pp)\n    vessel_results[vessel_name] = result\n    current_solution = result['treated_solution']\n    \n    logger.info(f\"  - Breakthrough: {result['breakthrough_bv']} BV\")\n    logger.info(f\"  - Hardness out: {result['total_hardness_mg_L']:.1f} mg/L as CaCO3\")\n    logger.info(f\"  - pH out: {result['pH']:.2f}\")

In [ ]:
# Compile simulation results\nfinal_solution = current_solution\n\n# Build treated water composition\ntreated_water = {\n    'flow_m3_hr': water_analysis['flow_m3_hr'],\n    'temperature_celsius': final_solution.T,\n    'pressure_bar': 1.0,\n    'pH': final_solution.pH,\n    'ion_concentrations_mg_L': {}\n}\n\n# Map back to original ion names\nreverse_mapping = {\n    'Ca': 'Ca_2+',\n    'Mg': 'Mg_2+',\n    'Na': 'Na_+',\n    'K': 'K_+',\n    'Cl': 'Cl_-',\n    'S(6)': 'SO4_2-',\n    'C(4)': 'HCO3_-',\n    'N(5)': 'NO3_-'\n}\n\nfor element, ion in reverse_mapping.items():\n    try:\n        if element == 'S(6)':\n            conc = final_solution.total('S(6)') * 96.06  # SO4 MW\n        elif element == 'C(4)':\n            conc = final_solution.total('HCO3-') * 61.02  # HCO3 MW\n        else:\n            conc = final_solution.total(element) * pp.get_element(element).mass\n        treated_water['ion_concentrations_mg_L'][ion] = conc\n    except:\n        treated_water['ion_concentrations_mg_L'][ion] = 0\n\n# Build IX performance metrics\nix_performance = {}\nfor vessel_name, result in vessel_results.items():\n    flow_rate_L_hr = water_analysis['flow_m3_hr'] * 1000\n    vessel_config = configuration['ix_vessels'][vessel_name]\n    resin_volume_L = vessel_config['resin_volume_m3'] * 1000\n    \n    breakthrough_time = result['breakthrough_bv'] * resin_volume_L / flow_rate_L_hr\n    \n    ix_performance[vessel_name] = {\n        'breakthrough_time_hours': breakthrough_time,\n        'bed_volumes_treated': result['breakthrough_bv'],\n        'regenerant_consumption_kg': vessel_config['resin_volume_m3'] * vessel_config.get('regenerant_dose_kg_m3', 100),\n        'capacity_utilization_percent': min(100, result['breakthrough_bv'] / 1000 * 100),\n        'total_throughput_m3': result['breakthrough_bv'] * vessel_config['resin_volume_m3']\n    }\n\n# Generate recommendations\nrecommendations = []\nmin_breakthrough_vessel = min(vessel_results.items(), key=lambda x: x[1]['breakthrough_bv'])\nrecommendations.append(f\"{min_breakthrough_vessel[0]} will breakthrough first at {min_breakthrough_vessel[1]['breakthrough_bv']} BV\")\n\nif flowsheet_type == 'nawac_degasser':\n    recommendations.append(\"Monitor alkalinity reduction to prevent over-softening\")\nelif flowsheet_type == 'hwac_degasser':\n    recommendations.append(\"Ensure adequate degassing to raise pH above 6.5\")\nelif flowsheet_type == 'sac_flowsheet':\n    recommendations.append(\"Monitor Na+ increase in product water\")\n\n# Final result structure\nresult = {\n    'status': 'success',\n    'watertap_notebook_path': 'executed_in_notebook',\n    'model_type': simulation_options.get('model_type', 'phreeqc'),\n    'actual_runtime_seconds': 1.0,\n    'treated_water': treated_water,\n    'ix_performance': ix_performance,\n    'water_quality_progression': [],  # Could be populated with time-series data\n    'economics': configuration.get('economics', {}),\n    'recommendations': recommendations\n}\n\nif degasser_result:\n    result['degasser_performance'] = degasser_result\n\nlogger.info(\"\\nSimulation complete!\")\nlogger.info(f\"Final water quality: pH {treated_water['pH']:.2f}, hardness {treated_water['ion_concentrations_mg_L'].get('Ca_2+', 0) + treated_water['ion_concentrations_mg_L'].get('Mg_2+', 0):.1f} mg/L\")